<a href="https://colab.research.google.com/github/TheOctoMizer/AAI-510-Project/blob/main/FinalProjectSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sentimenet Analysis using multilingual texts:
